In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

2023-03-08 13:51:53.585352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 13:51:54.912691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 13:51:54.912767: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 13:51:57.874781: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

2023-03-08 13:52:03.291730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-08 13:52:03.292159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 13:52:03.292296: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-08 13:52:03.292481: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-08 13:52:03.292565: W tensorf

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

#load pandas dataframe (rimuovo le righe vuote che putroppo sono presenti)
vax_series = pd.read_csv('data/posts_ULTRAcleaned_it_only_spacy.csv')
vax_series.dropna(inplace=True)

#salviamo una lista di lista con le parole tokenizzate
vax_text = vax_series['clean_text'].tolist()

vax_text


['pronto affrontare settimana quirinale booster',
 'pass sanitaire guarigione booster pezzo dose migliore dellimmunite covid volere togliere scadenza spg tridosato autorizzare infettare natural arrestatele',
 'vaccinare bambino ricevere almeno dose Italia booster over piano notizia vaccare',
 'sereno sbrocchiamo periodo perch certezza curare grare riconoscente vaccinare booster fatelo vaccino obbligatorio',
 'vaccinare tredoso intero famiglia positivo covid influencer giornalista politico virologo commento proposito Greenpass booster tampono sivax novax emergenza',
 'giovane dovere dose booster unico scudo omicron',
 'booster',
 'continuare ce fregare draghi Milan Juventus inter Venezia quirinale quirinale draghistan vaccino obbligatorio vaccino covid novax dittatura drago vattenere serie tim amico booster greenpassre',
 'esperto israele chiedere somministrazione quarto dose vaccare over proteggere complicazione pi dimezzare rischio contagio covid israel booster',
 'inter inzago positi

In [4]:
import spacy
import pandas as pd
import regex as re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
it_stopwords = stopwords.words('italian')

#the stemmer requires a language parameter
snow_stemmer = SnowballStemmer(language='italian')

def cleaner(s):
    #removing numbers and special caracters
    s = re.sub(r'[^a-z\s]', '', s).strip()
    #remving multiple spaces
    s = " ".join(s.split()).strip()
    #some manual corrections
    #s = s.replace(' accino', ' vaccino')
    #removing stopwords with nltk
    s = stop_word_remover(s)
    #stemming with nltk
    s = stemmer(s)

    return s


def stop_word_remover(text):
    word_tokens = word_tokenize(text)
    word_list = [word for word in word_tokens if word.lower() not in it_stopwords]
    return " ".join(word_list)

def stemmer(text):
    words = word_tokenize(text)
    stem_words = [snow_stemmer.stem(word) for word in words]
    return " ".join(stem_words)

query = "Secondo tutte le leggi conosciute dell’aeronautica, non c’è modo che un ape sia in grado di volare. Le sue ali sono troppo piccole per sollevare il suo corpicino grassoccio da terra. L’ape, si sa, vola lo stesso. Perché alle api non interessa quello che gli uomini ritengono impossibile"
#query = "vaccino pfizer lanciato sul mercato: 1 morto e 3 feriti"
#query=cleaner(query)

print(query)


Secondo tutte le leggi conosciute dell’aeronautica, non c’è modo che un ape sia in grado di volare. Le sue ali sono troppo piccole per sollevare il suo corpicino grassoccio da terra. L’ape, si sa, vola lo stesso. Perché alle api non interessa quello che gli uomini ritengono impossibile


In [5]:
sentence_embeddings = model(vax_text)

query_vec = model([query])[0]

In [16]:
from scipy.spatial.distance import cosine, euclidean

for sent in vax_text[:10]:
  sim = 1 - cosine(query_vec, model([sent])[0])
  #sim = cosine(query_vec, sent)
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  pronto affrontare settimana quirinale booster ; similarity =  0.1998051404953003
Sentence =  pass sanitaire guarigione booster pezzo dose migliore dellimmunite covid volere togliere scadenza spg tridosato autorizzare infettare natural arrestatele ; similarity =  0.3452955186367035
Sentence =  vaccinare bambino ricevere almeno dose Italia booster over piano notizia vaccare ; similarity =  0.35866230726242065
Sentence =  sereno sbrocchiamo periodo perch certezza curare grare riconoscente vaccinare booster fatelo vaccino obbligatorio ; similarity =  0.44728201627731323
Sentence =  vaccinare tredoso intero famiglia positivo covid influencer giornalista politico virologo commento proposito Greenpass booster tampono sivax novax emergenza ; similarity =  0.4702686369419098
Sentence =  giovane dovere dose booster unico scudo omicron ; similarity =  0.45410382747650146
Sentence =  booster ; similarity =  0.1271335780620575
Sentence =  continuare ce fregare draghi Milan Juventus inte

In [13]:
vax_text

['pronto affrontare settimana quirinale booster',
 'pass sanitaire guarigione booster pezzo dose migliore dellimmunite covid volere togliere scadenza spg tridosato autorizzare infettare natural arrestatele',
 'vaccinare bambino ricevere almeno dose Italia booster over piano notizia vaccare',
 'sereno sbrocchiamo periodo perch certezza curare grare riconoscente vaccinare booster fatelo vaccino obbligatorio',
 'vaccinare tredoso intero famiglia positivo covid influencer giornalista politico virologo commento proposito Greenpass booster tampono sivax novax emergenza',
 'giovane dovere dose booster unico scudo omicron',
 'booster',
 'continuare ce fregare draghi Milan Juventus inter Venezia quirinale quirinale draghistan vaccino obbligatorio vaccino covid novax dittatura drago vattenere serie tim amico booster greenpassre',
 'esperto israele chiedere somministrazione quarto dose vaccare over proteggere complicazione pi dimezzare rischio contagio covid israel booster',
 'inter inzago positi

In [7]:
import numpy as np
# k-means - Inertia as the number of clusters varies
from sklearn.cluster import KMeans
from sklearn import preprocessing  # to normalise existing X

#create a numpy array with all the vectors
vectors = np.zeros((len(sentence_embeddings), 512))
for i in range(len(sentence_embeddings)):
    vectors[i] = sentence_embeddings[i]

#normalize
vectors = preprocessing.normalize(vectors)

X_train_PCA = pd.DataFrame(vectors)

n_clusters = 2
n_init = 10
max_iter = 300
tol = 0.0001
random_state = 2018
# n_jobs = 2
kMeans_inertia = pd.DataFrame(data=[],index=range(2), \
columns=['inertia'])

kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, \
    max_iter=max_iter, tol=tol, random_state=random_state )
   
kmeans.fit(X_train_PCA)

center = kmeans.cluster_centers_
center.shape

(2, 512)

In [21]:
#calculating the distance from the 2 centers
from gensim import similarities
from scipy.spatial.distance import cosine, euclidean
import spacy
import pandas as pd
import regex as re
import numpy as np


#definiziamo una lista di frasi che dovrebbero essere classificate come anomalie
anomalies = ["il cane gioca a palla","ho dovuto ripristinare il PC per togliere tutti i suoi malware", "bella giornata oggi" ]
novax_sentences = ["i vaccini uccidono", "non mi voglio vaccinare", "no al booster", "dopo 4 dosi il sistema immunitario fritto", "i vaccini fanno morire le persone anche se ci dicono di no"]
#provo con un testo casuale
#testo = "i vaccini fanno morire le persone anche se ci dicono di no"
#test_vector = sbert_model.encode([anomalies])
#test_vector = model(testo)

cosine_distance = []
euclidean_distance = []
'''cosine_distance.append(  cosine(test_vector[0],center[0]) )
cosine_distance.append(  cosine(test_vector[0],center[1]) )
euclidean_distance.append (euclidean(test_vector[0],center[0]))
euclidean_distance.append(euclidean(test_vector[0],center[1]))

print("Distance score cosine 1 - 2:",cosine_distance[0],cosine_distance[1])
print("Distance score euclidean 1 - 2:",euclidean_distance[0],euclidean_distance[1])
print ("Euclidean distance sum:",sum(euclidean_distance))'''

print("Anomalies score:")
for i in anomalies:
   
    test_vector = model([i])

    cosine_distance = []
    euclidean_distance = []

    cosine_distance.append(  cosine(test_vector[0],center[0]) )
    cosine_distance.append(  cosine(test_vector[0],center[1]) )
    euclidean_distance.append (euclidean(test_vector[0],center[0]))
    euclidean_distance.append(euclidean(test_vector[0],center[1]))
   
    print ("Euclidean distance sum:",cosine_distance[0]+cosine_distance[1])

print("Novax score:")
for i in novax_sentences:
    test_vector = model([i])

    cosine_distance = []
    euclidean_distance = []

    cosine_distance.append(  cosine(test_vector[0],center[0]) )
    cosine_distance.append(  cosine(test_vector[0],center[1]) )
    euclidean_distance.append (euclidean(test_vector[0],center[0]))
    euclidean_distance.append(euclidean(test_vector[0],center[1]))
   
    print ("Euclidean distance sum:",cosine_distance[0]+cosine_distance[1])

Anomalies score:
Euclidean distance sum: 0.9886950861360096
Euclidean distance sum: 1.0876352980511506
Euclidean distance sum: 1.04280290646287
Novax score:
Euclidean distance sum: 1.7469873493631871
Euclidean distance sum: 0.9513516025978438
Euclidean distance sum: 1.3739042161596244
Euclidean distance sum: 0.9287548250062411
Euclidean distance sum: 1.0126217617669886


In [18]:
[anomalies[1]]

['ho dovuto ripristinare il PC per togliere tutti i suoi malware']